<a href="https://colab.research.google.com/github/EdgarTeran77/Proyectos/blob/main/DeberDispositivosTema%3ASucidios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install beautifulsoup4 requests pandas

In [102]:
import json # modulo para trabajar con archivos JSON
import requests # libreria para hacer peticiones HTTP
import pandas as pd # libreria para procesar datos, proporciona funciones para analíticas
from bs4 import BeautifulSoup as bs # Extraer información de paginas web

In [103]:
def rowsToDataFrame(rows):
    df = pd.DataFrame(rows[1:], columns=rows[0])
    return df

In [104]:
def processTableData(tbl):
    rows = []
    for child in tbl.find('tbody').children: # itera los items internos de la tabla [<tr>]
        row = []
        for td in child: # itera los items internos de la tabla [<td>]
            try:
                item = td.text.replace('\n', '') # recupera el texto del elemento [<td>]
                if item:
                    row.append(item) # Agrega cada elemento de la fila
            except:
                continue
        if len(row) > 0:
            rows.append(row) # agrega todos los campos de una fila de la tabla
    # print(rows)
    return rows

In [185]:
def processDataHTML(data):
    try:
        soup = bs(data['sections'][2]['text'], 'html.parser') # Recupera el item del JSON y lo convierte en contenido HTML
        tables = soup.find_all('table')

        if not tables:
            raise ValueError("No se encontraron tablas en la página.")

        tbl = tables[0] # Recupera la primera tabla del contenido HTML
        tblRows = processTableData(tbl) # Ejecuta la función para procesar el contenido de la Tabla
        return tblRows

    except Exception as e:
        print(f"Error al procesar el HTML: {e}")
        return None



In [187]:
# Read website
# r = requests.get('https://www.wikiwand.com/es/Anexo:Tabla_estadística_de_la_Copa_Mundial_de_Fútbol')
r = requests.get("https://www.wikiwand.com/mcs-api/es.wikipedia.org/v1/page/mobile-sections-remaining/Anexo%3APa%C3%ADses_por_tasa_de_suicidio")  # Recuperamos el contenido de la web
# print(r)
table = processDataHTML(json.loads(r.content)) # Formate el contenido en formato JSON
df = rowsToDataFrame(table) # ejecuta la funcion para recupera los datos de la tabla como dataframe de Pandas
df

,Regiones,Masculino,Femenino,Promedio,Año
0,Groenlandia,"84,99","28,44","58,28 (0,0583%)",2016
1,Lituania,"65,1","12,4","28,27 (0,02827%)",2016
2,Corea del Sur,"38,5","14,8","26,6 (0,0266%)",2018
3,Guyana,"41,25","10,20","25,52 (0,0255%)",2017
4,Kazajistán,"40,68","8,01","23,81 (0,0238%)",2017
...,...,...,...,...,...
107,Granada,"0,0","0,0","0,0 (0%)",2008
108,San Cristóbal y Nieves,"0,0","0,0","0,0 (0%)",1995
109,Antigua y Barbuda,"0,0","0,0","0,0 (0%)",1995
110,Haití,"0,0","0,0","0,0 (0%)",2003


In [188]:
df.to_csv('tabla.csv', index=False)

In [92]:
!pip install pandas seaborn matplotlib scikit-learn

In [189]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # Para graficación
from sklearn.cluster import KMeans # Para el modelamiento

In [190]:
url = 'tabla.csv' # external url?

In [191]:
data = pd.read_csv(url) # Recupera los datos en formato csv a tipo dataframe (Estructura de datos bidimensional)
data.head() # Visualiza los 5 primeros datos del dataframe

,Regiones,Masculino,Femenino,Promedio,Año
0,Groenlandia,"84,99","28,44","58,28 (0,0583%)",2016
1,Lituania,"65,1","12,4","28,27 (0,02827%)",2016
2,Corea del Sur,"38,5","14,8","26,6 (0,0266%)",2018
3,Guyana,"41,25","10,20","25,52 (0,0255%)",2017
4,Kazajistán,"40,68","8,01","23,81 (0,0238%)",2017


In [192]:
# Visualizamos el DataFrame
data

,Regiones,Masculino,Femenino,Promedio,Año
0,Groenlandia,"84,99","28,44","58,28 (0,0583%)",2016
1,Lituania,"65,1","12,4","28,27 (0,02827%)",2016
2,Corea del Sur,"38,5","14,8","26,6 (0,0266%)",2018
3,Guyana,"41,25","10,20","25,52 (0,0255%)",2017
4,Kazajistán,"40,68","8,01","23,81 (0,0238%)",2017
...,...,...,...,...,...
107,Granada,"0,0","0,0","0,0 (0%)",2008
108,San Cristóbal y Nieves,"0,0","0,0","0,0 (0%)",1995
109,Antigua y Barbuda,"0,0","0,0","0,0 (0%)",1995
110,Haití,"0,0","0,0","0,0 (0%)",2003


In [193]:
# Inspeccionamos el tipo de dato de cada variable
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Regiones   112 non-null    object
 1   Masculino  112 non-null    object
 2   Femenino   112 non-null    object
 3   Promedio   77 non-null     object
 4   Año        77 non-null     object
dtypes: object(5)
memory usage: 4.5+ KB


In [194]:
data.describe()

,Regiones,Masculino,Femenino,Promedio,Año
count,112,112,112,77,77
unique,112,98,55,57,21
top,Groenlandia,"0,0",2016,"0,0 (0%)",2008
freq,1,6,21,5,16


In [195]:
def fixTitulos(val):
    return 0 if val == '-' else int(val)

In [196]:
def fixRend(val):
    if (val == '%'):
        return float(val.replace(',', '.').replace('%', ''))
    return val

In [197]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Regiones   112 non-null    object
 1   Masculino  112 non-null    object
 2   Femenino   112 non-null    object
 3   Promedio   77 non-null     object
 4   Año        77 non-null     object
dtypes: object(5)
memory usage: 4.5+ KB


In [198]:
def fixTitulos(val):
    return 0 if val == '-' else int(val)

In [202]:
def fixRend(val):
    if (val == '%'):
        return float(val.replace(',', '.').replace('%', ''))
    return val

In [203]:
# Inspeccionamos el tipo de dato de cada variable
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Regiones   112 non-null    object
 1   Masculino  112 non-null    object
 2   Femenino   112 non-null    object
 3   Promedio   77 non-null     object
 4   Año        77 non-null     object
dtypes: object(5)
memory usage: 4.5+ KB


In [204]:
data[['Masculino','Femenino', 'Promedio', 'Año']].corr().style.background_gradient(cmap='coolwarm').set_precision(2)

<ipython-input-204-bb1e78326cb0>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data[['Masculino','Femenino', 'Promedio', 'Año']].corr().style.background_gradient(cmap='coolwarm').set_precision(2)
<ipython-input-204-bb1e78326cb0>:1: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  data[['Masculino','Femenino', 'Promedio', 'Año']].corr().style.background_gradient(cmap='coolwarm').set_precision(2)
